In [1]:
import json
import pandas as pd
from utils import prep_ans
import numpy as np
from vqa_question_type_processing import question_type_processing as vqa_question_type_processing
from simpsons_question_type_processing import question_type_processing as simpsons_question_type_processing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from io import BytesIO
from PIL import Image
import numpy as np
import wandb
import matplotlib.pyplot as plt
import seaborn as sns
import wandb

In [2]:
vqav2_anno_paths = ["/home/ngoc/data/vqav2/new_vqa2_test_annotation.json"]

In [3]:
vqa_ans_to_idx, idx_to_vqa_ans, question_type_map = json.load(open("/home/ngoc/githubs/chatgpt-hie-vqa/datasets/answer_dicts_vqav2.json", 'r'))

In [38]:
def map_prompt(question, answer):

    prompt = (f'Classify a question-asnwer pair into yes/no, object, color, number, location, human, action, sport, other:\n'
                f'Question: What is the capital of France?.\n'
                f'Answer: Paris.\n'
                f'Type: location\n\n'
                f'Question: {question}.\n'
                f'Answer: {answer}.\n'
                f'Type:')
    
    return prompt

In [4]:
vqa_question_type_dict = {}
with open("/home/ngoc/githubs/Question-Analysis/datasets/train_question_type_gpt_v6_vqav2.json", 'r') as file:
    for line in file:
        question_object = json.loads(line)
        question_str = question_object["question"]
        question_type = question_object["question_type"]
        question_type = vqa_question_type_processing(question_type)
        vqa_question_type_dict[question_str] = question_type
file.close()


vqa_question_paths = [ "/home/ngoc/data/vqav2/new_vqa2_test_question.json"]
vqa_questions = []
for vqa_question_path in vqa_question_paths:
    with open(vqa_question_path, 'r') as file:
        vqa_questions += json.load(file)
        
        
vqa_processed_questions = {}
for question in vqa_questions:
    # print(question)
    # print(vqa_question_type_dict[question["question"])
    # a
    vqa_processed_questions[question["question_id"]] = {
        "question_type": vqa_question_type_dict[question["question"]],
        "question": question["question"]
        }


In [5]:

vqav2_answers = []
for vqav2_anno_path in vqav2_anno_paths:
    with open(vqav2_anno_path, 'r') as file:
        vqav2_answers += json.load(file)

def proc_ans(ans, ans_to_ix):
    ans_prob_dict = {}
    for ans_ in ans['answers']:
        ans_proc = prep_ans(ans_['answer'])
        if ans_proc not in ans_prob_dict:
            ans_prob_dict[ans_proc] = 1
        else:
            ans_prob_dict[ans_proc] += 1
    answer_scores = {} 
    for ans_ in ans_prob_dict:
        if ans_ in ans_to_ix:
            answer_scores[ans_] = get_score(ans_prob_dict[ans_])
    return answer_scores

def get_score(occur):
    if occur == 0:
        return .0 
    elif occur == 1:
        return round(1/3,2)
    elif occur == 2:
        return round(2/3,2)
    elif occur >= 3:
        return 1

vqav2_processed_answers = {}
for vqav2_answer in  vqav2_answers:
    question_id = vqav2_answer["question_id"]
    vqav2_processed_answers[question_id] = {}
    multiple_choice_answer =  prep_ans(vqav2_answer['multiple_choice_answer'])
    process_answers = proc_ans(vqav2_answer, vqa_ans_to_idx)
    vqav2_processed_answers[question_id]["answer_dict"] = process_answers
    vqav2_processed_answers[question_id]["multiple_choice_answer"] = multiple_choice_answer
    vqav2_processed_answers[question_id]["question_type"] = vqa_processed_questions[question_id]["question_type"]
    vqav2_processed_answers[question_id]["question"] = vqa_processed_questions[question_id]["question"]
    

In [12]:
test_result = pd.read_csv("test_predictions.csv")
test_result["multiple_choice_answer"] = test_result["question_id"].apply(lambda x: vqav2_processed_answers[x]["multiple_choice_answer"])
test_result = test_result[test_result["multiple_choice_answer"].isin(vqa_ans_to_idx.keys())]

In [16]:
question_answer_pair_type = pd.read_csv("vqa_test_qa_type.csv")
question_answer_pair_type = question_answer_pair_type.rename(columns={"answer": "multiple_choice_answer"})
question_answer_pair_type["vqa_prediction"] = question_answer_pair_type["multiple_choice_answer"]
question_answer_pair_type.head()

,question,multiple_choice_answer,qa_type,vqa_prediction
0,What is this photo taken looking through?,net,object,net
1,What position is this man playing?,pitcher,sport,pitcher
2,What color is the players shirt?,orange,color,orange
3,Is this man a professional baseball player?,yes,yes/no,yes
4,What color is the snow?,white,color,white


In [17]:
def map_answer_scores(question_id, vqa_prediction):
    score_dict = vqav2_processed_answers[question_id]["answer_dict"]
    if vqa_prediction in score_dict.keys():
        return score_dict[vqa_prediction]
    else:
        return 0

In [18]:
try:
    test_result["vqa_prediction"] = test_result["prediction"]
except:
    pass  

In [19]:
test_result["vqa_score"] = test_result[["question_id", "vqa_prediction"]].apply(lambda x: map_answer_scores(x["question_id"], x["vqa_prediction"]), axis = 1)
test_result["question_type"] = test_result["question_id"].apply(lambda x: vqav2_processed_answers[x]["question_type"])
test_result["question"] = test_result["question_id"].apply(lambda x: vqav2_processed_answers[x]["question"])


In [20]:
print(f"len of test set: {len(test_result)}")

len of test set: 94499


In [21]:
test_result= pd.merge(test_result, question_answer_pair_type[['question', 'multiple_choice_answer', 'qa_type']], on=['question', 'multiple_choice_answer'], how='inner')
test_result = test_result.rename(columns ={'qa_type': 'qa_type_label'})
print(f"len of test set: {len(test_result)}")

len of test set: 94483


In [22]:
test_result = pd.merge(test_result, question_answer_pair_type[['question', 'vqa_prediction', 'qa_type']], on=['question', 'vqa_prediction'], how='left')
test_result = test_result.rename(columns ={'qa_type': 'qa_type_prediction'})

In [39]:
if test_result["qa_type_prediction"].isna().any():
    
    test_result_non_qa_type = test_result[test_result["qa_type_prediction"].isna()]
    test_result_non_qa_type["answer"] = test_result_non_qa_type["vqa_prediction"]
    test_result_non_qa_type = test_result_non_qa_type[["answer", "question"]]
    test_result_non_qa_type = test_result_non_qa_type.drop_duplicates()
    test_result_non_qa_type = test_result_non_qa_type.reset_index(drop=True)
    test_result_non_qa_type["prompt"] = test_result_non_qa_type[["question", "answer"]].apply(lambda x: map_prompt(x["question"], x["answer"]), axis =1)
    print(f"number of new qa pairs {len(test_result_non_qa_type)}")
    test_result_non_qa_type.to_csv("new_qa_pairs_for_evaluation.csv", index="False")
    raise Exception("Existing QA pairs don't have a type")

/tmp/ipykernel_645361/3088051369.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_result_non_qa_type["answer"] = test_result_non_qa_type["vqa_prediction"]


number of new qa pairs 33317


Exception: Existing QA pairs don't have a type

In [ ]:
test_result[""]

,question_id,prediction,multiple_choice_answer,vqa_prediction,vqa_score,question_type,question,qa_type_label,qa_type_prediction
0,300383000,black,white,black,0.00,color,What color are the tennis shoes?,color,NaN
2,290189005,north,left,north,0.00,location,Which direction is the train going?,other,NaN
4,119604001,yellow,orange,yellow,1.00,color,What is the main color of the truck?,color,NaN
6,4647001,black,white,black,0.00,color,What color is the duck's beak?,color,NaN
15,545285003,solid,0,solid,0.67,object,What pattern is on the seat cushions?,object,NaN
...,...,...,...,...,...,...,...,...,...
94469,406462005,beach,desert,beach,0.67,location,Where does the horse race take place?,location,NaN
94475,421763000,apples,oranges,apples,0.00,object,Which fruits are these?,object,NaN
94477,36953015,living room,kitchen,living room,0.00,yes/no,Is the room on the left a kitchen or a breakroom?,object,NaN
94478,508311002,eating,standing,eating,0.33,action,What are these birds doing?,action,NaN


In [28]:
test_overall_accuracy  = test_result["vqa_score"].sum()/len(test_result)

In [29]:
sorted_unique_labels = ["yes/no", "object", "color", "number", "location","human", "action", "sport", "other"]

In [30]:
test_overall_accuracy

0.5500317517436999

In [ ]:
sub_accurancy = {}
for question_type_value in sorted_unique_labels:
    sub_test_result = test_result[test_result["question_label"] == question_type_value]
    sub_accuracy_i = sub_test_result["vqa_score"].sum()/len(sub_test_result)
    sub_accurancy["test_vqa_accuracy_" + question_type_value] = sub_accuracy_i

In [ ]:
sub_accurancy

{'test_vqa_accuracy_yes/no': 0.6705244693021483,
 'test_vqa_accuracy_object': 0.45501823178142997,
 'test_vqa_accuracy_color': 0.45560325810283386,
 'test_vqa_accuracy_number': 0.37053732602277517,
 'test_vqa_accuracy_location': 0.5301192472517235,
 'test_vqa_accuracy_human': 0.4644654498044329,
 'test_vqa_accuracy_action': 0.5544229607250755,
 'test_vqa_accuracy_sport': 0.7490697674418604,
 'test_vqa_accuracy_other': 0.46175619047619043}

In [ ]:
run_id = "bi6l2w1x"

In [ ]:

wandb.init(project="VQA new", id=run_id, resume="must")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ngocdunghuynh. Use `wandb login --relogin` to force relogin


In [ ]:
wandb.log({"test_vqa_accuracy": test_overall_accuracy})

In [ ]:
wandb.log(sub_accurancy)

In [ ]:
test_question_type_accuracy = classification_report(test_result["question_prediction"], test_result["question_label"], output_dict=True)

In [ ]:
new_test_question_type_accuracy = {}
for key in test_question_type_accuracy:
    new_test_question_type_accuracy["question_type_" + key] = test_question_type_accuracy[key]
del(test_question_type_accuracy)

In [ ]:
wandb.log(new_test_question_type_accuracy)

In [ ]:
y_true = test_result['question_label']
y_pred = test_result['question_prediction']
sorted_unique_labels = ["yes/no", "object", "color", "number", "location","human", "action", "sport", "other"]

conf_matrix = confusion_matrix(y_true, y_pred, labels=sorted_unique_labels)
conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(20, 9))
plt.subplot(1, 2, 1)
sns.heatmap(conf_matrix, annot=True, fmt='d', 
            xticklabels=sorted_unique_labels, yticklabels=sorted_unique_labels, 
            cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Type')
plt.ylabel('True Type')
plt.subplot(1, 2, 2)
sns.heatmap(conf_matrix_normalized, annot=True, fmt='.2f', 
            xticklabels=sorted_unique_labels, yticklabels=sorted_unique_labels, 
            cmap='Blues', cbar=False)
plt.title('Normalized Confusion Matrix')
plt.xlabel('Predicted Type')
plt.ylabel('True Type')
plt.suptitle(f'Test Confusion Matrix')
buffer = BytesIO()
plt.savefig(buffer, format='png')
buffer.seek(0)
image = Image.open(buffer)
image_array = np.array(image)
wandb.log({"Test Confusion Matrix": wandb.Image(image_array)})
plt.close()
plt.show()

In [ ]:
wandb.finish()

wandb: WARNING Ensure read and write access to run files dir: /home/ngoc/githubs/chatgpt-hie-vqa/result_analysis/wandb/run-20240325_152136-bi6l2w1x/files, control this via the WANDB_DIR env var. See https://docs.wandb.ai/guides/track/environment-variables


question_type_accuracy,▁
test_vqa_accuracy,▁
test_vqa_accuracy_action,▁
test_vqa_accuracy_color,▁
test_vqa_accuracy_human,▁
test_vqa_accuracy_location,▁
test_vqa_accuracy_number,▁
test_vqa_accuracy_object,▁
test_vqa_accuracy_other,▁
test_vqa_accuracy_sport,▁
test_vqa_accuracy_yes/no,▁
